<a href="https://colab.research.google.com/github/rsaketh002/100-nlp-papers/blob/master/Natural%20Language%20Processing%20with%20Disaster%20Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"gandeedsaketh","key":"05c8f8f5ff51d6f5a5f7d7f452b59742"}'}

In [3]:
 ! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

In [4]:
 ! chmod 600 ~/.kaggle/kaggle.json

In [5]:
 ! kaggle datasets list

ref                                                                title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
nelgiriyewithana/apple-quality                                     Apple Quality                                      170KB  2024-01-11 14:31:07           6855        164  1.0              
saimondahal/life-expectancy-trends-for-males-and-females           Life Expectancy Trends for Males and Females       263KB  2024-01-28 09:26:09            784         28  1.0              
hummaamqaasim/jobs-in-data                                         Jobs and Salaries in Data Science                   76KB  2023-12-25 20:03:32          13091        240  1.0              
yashusinghal/electric-vehicle-population-dataset  

In [6]:
!kaggle competitions download -c nlp-getting-started

100% 593k/593k [00:00<00:00, 740kB/s]
100% 593k/593k [00:00<00:00, 740kB/s]


In [7]:
! mkdir train

In [8]:
! unzip /content/nlp-getting-started.zip -d train

Archive:  /content/nlp-getting-started.zip
  inflating: train/sample_submission.csv  
  inflating: train/test.csv          
  inflating: train/train.csv         


In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [10]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.1 MB/s eta 0:00:00


In [11]:
df = pd.read_csv('/content/train/train.csv')

In [18]:
df.head()

,id,keyword,location,text,target
0,1,none,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,none,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,none,NaN,All residents asked to 'shelter in place' are ...,1
3,6,none,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,none,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [13]:
checkpoint='xlm-roberta-large'

In [14]:
df.shape

(7613, 5)

In [17]:
df['keyword'].fillna('none',inplace=True)

In [21]:
df.drop(['id','keyword','location'],axis=1,inplace=True)

In [22]:
df.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [23]:
from transformers import AutoTokenizer,TFAutoModelForSequenceClassification

In [24]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [25]:
from datasets import load_dataset

In [26]:
from datasets import Dataset

In [37]:
df.columns=['text','label']

In [38]:
dataset = Dataset.from_pandas(df)

In [39]:
dataset=dataset.train_test_split(test_size=0.2,shuffle=True)

In [40]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 6090
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1523
    })
})

In [41]:
def preprocess(ans):
  return tokenizer(ans['text'],truncation=True)

In [42]:
tokenized_dataset = dataset.map(preprocess, batched=True)

Map:   0%|          | 0/6090 [00:00<?, ? examples/s]

Map:   0%|          | 0/1523 [00:00<?, ? examples/s]

In [43]:
tokenized_dataset=tokenized_dataset.remove_columns('text')

In [44]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 6090
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 1523
    })
})

In [45]:
from transformers import DataCollatorWithPadding

In [46]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer,return_tensors='tf')

In [47]:
from sklearn.metrics import accuracy_score,f1_score

In [48]:
def compute_metrics(eval_pred):
  predictions,labels=eval_pred
  predictions=np.argmax(predictions,axis=1)
  accuracy=accuracy_score(labels,predictions)
  f1=f1_score(labels,predictions,average='weighted')
  return {'accuracy':accuracy,'f1':f1}

In [50]:
id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative": 0, "Positive": 1}

In [51]:
from transformers import create_optimizer

batch_size=32
epochs=3
batches_per_epoch=len(tokenized_dataset['train'])//batch_size
total_train_steps=int(batches_per_epoch*epochs)
optimizer,schedule=create_optimizer(init_lr=2e-5,num_train_steps=total_train_steps,num_warmup_steps=0)

In [53]:
model=TFAutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=2,id2label=id2label,label2id=label2id)

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFXLMRobertaForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFXLMRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [54]:
tf_train_dataset=model.prepare_tf_dataset(tokenized_dataset['train'],shuffle=True,batch_size=batch_size,collate_fn=data_collator)

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [55]:
val_dataset=model.prepare_tf_dataset(tokenized_dataset['test'],shuffle=False,batch_size=batch_size,collate_fn=data_collator)

In [56]:
next(iter(tf_train_dataset))

({'input_ids': <tf.Tensor: shape=(32, 68), dtype=int64, numpy=
  array([[    0,  1374, 22769, ...,     1,     1,     1],
         [    0,   310, 25585, ...,     1,     1,     1],
         [    0,  1374,   683, ...,     1,     1,     1],
         ...,
         [    0, 80392,   214, ...,     1,     1,     1],
         [    0,    62, 10176, ...,     1,     1,     1],
         [    0,    17,   839, ...,     1,     1,     1]])>,
  'attention_mask': <tf.Tensor: shape=(32, 68), dtype=int64, numpy=
  array([[1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         ...,
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0]])>},
 <tf.Tensor: shape=(32,), dtype=int64, numpy=
 array([0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1,
        1, 1, 1, 1, 1, 1, 0, 0, 0, 1])>)

In [57]:
model.compile(optimizer=optimizer)

In [58]:
from transformers.keras_callbacks import KerasMetricCallback

In [59]:
metric_callback=KerasMetricCallback(compute_metrics,eval_dataset=val_dataset)

In [60]:
callbacks=[metric_callback]

In [61]:
model.fit(tf_train_dataset,epochs=epochs,callbacks=callbacks,validation_data=val_dataset)

Epoch 1/3
190/190 [==============================] - 183s 517ms/step - loss: 0.5418 - val_loss: 0.5203 - accuracy: 0.7873 - f1: 0.7884
Epoch 2/3
190/190 [==============================] - 72s 380ms/step - loss: 0.3977 - val_loss: 0.3735 - accuracy: 0.8437 - f1: 0.8424
Epoch 3/3
190/190 [==============================] - 71s 375ms/step - loss: 0.3378 - val_loss: 0.3961 - accuracy: 0.8365 - f1: 0.8356


In [62]:
test_df=pd.read_csv('/content/train/test.csv')

In [63]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [64]:
test_df.drop(['keyword','location'],axis=1,inplace=True)

In [65]:
test_df=Dataset.from_pandas(test_df)

In [66]:
test_df=test_df.map(preprocess,batched=False)

Map:   0%|          | 0/3263 [00:00<?, ? examples/s]

In [67]:
test_inputs=model.prepare_tf_dataset(test_df,shuffle=False,batch_size=1,collate_fn=data_collator)

In [68]:
predictions=model.predict(test_inputs)

3263/3263 [==============================] - 132s 39ms/step


In [70]:
predictions=tf.nn.softmax(predictions['logits'])

In [71]:
predictions = [np.argmax(i) for i in predictions]

In [72]:
result_df = pd.DataFrame({'id': test_df['id'], 'prediction': predictions})

In [73]:
result_df.head()

,id,prediction
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [74]:
result_df.to_csv('submission.csv', index=False)